In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from scipy.linalg import pinv2
from sklearn.metrics import accuracy_score
from numpy import linalg as LA
from sklearn.preprocessing import label_binarize
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import time
import pyswarms as ps

from pyswarms.single.global_best import GlobalBestPSO
from pyswarms.utils.functions import single_obj as fx
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
#Importing dataset
dataframe=pd.read_csv("ENB2012.csv")
dataframe.sample(frac=1) # Randomizing the whole dataset
DF_train,DF_test=train_test_split(dataframe, test_size=0.30) #Splitting the dataset into training and testing set with 70:30 ratio.
Tr_features=DF_train[['X1','X2','X3','X4','X5','X6','X7','X8']]
Te_features=DF_test[['X1','X2','X3','X4','X5','X6','X7','X8']]
Tr_Label=DF_train[['Y2']]
Te_Label=DF_test[['Y2']]

scalerObject=MinMaxScaler()
#Normalizing
Train_features=scalerObject.fit_transform(Tr_features)
Test_features=scalerObject.fit_transform(Te_features)
Train_Label=scalerObject.fit_transform(Tr_Label)
Test_Label=scalerObject.fit_transform(Te_Label)


In [3]:
def relu(X):
    return np.maximum(0,X)

In [4]:
def RMSE(y_actual,y_predicted):
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [5]:
def IELm_objective(dimensions,E):
  
    a = dimensions[0:8].reshape((len(Train_features[0]),1))
    b=dimensions[-1].reshape(1,1)
    predicted_accuracy=0  #Initial value of accuracy
    beta=0
    mul=np.dot(Train_features,a)
    sum=mul+b
    activation=relu(sum) # activation function sigmoid to calculate H
    beta_overall=np.dot(np.linalg.pinv(activation),E) # beta= inverse(H) * Target Label. As our target keeps on changing with new neuron
                                         #hence E
    y=np.dot(activation,beta_overall) # Calculated output
    E=E-y # Updated Training Label or Error
    predicted_rmse=RMSE(y,Train_Label) #Calculating the accuracy of the predicted labels
    return predicted_rmse

In [6]:
def swarm_initializer(input,**kwargs):

    particles = input.shape[0]
    for i in range(particles):
        loss = IELm_objective(input[i],**kwargs)
    return np.array(loss)

In [7]:
# Main Algorithm that increases the value of neurons in the hidden layer one by one and trying to achieve the best possible
# perfromance with the added neurons 

L=0 # Number of neurons in hidden layer inititally
max_neuron=50 # Maximum number of Neurons 
RMSE_expected=0.05  #Expected rmse or a threshold value that the network should have to break stop adding the neurons
predicted_rmse=100    #Initial value of rmse
beta=0
E=Train_Label  #Initial value of error as training set label, to keep a check on the differnce between the target and the predicted
new_a=np.random.uniform(low=-1,high=1,size=[len(Train_features[0]),1]) #Generating numpy array of weights for the first hidden neuron
new_b=np.random.uniform(low=-1,high=1,size=[1])
print(new_a)

# ****** TRAININIG STARTS HERE ***********
#Loop until the hidden layer reaches maximum number of neurons and until the predicted accuracy doesnt surpass the expected accuracy
start_time=time.time()

options = {'c1': 0.5, 'c2': 0.3, 'w':0.9} #Setting the hyper parameter and passing to the optimizer
lb = [-1,-1,-1,-1,-1,-1,-1,-1,-1] #setting the upper and lower limit bounds for the weights and biases
ub = [1,1,1,1,1,1,1,1,1]
bounds=(lb,ub)

while L<max_neuron and RMSE_expected<predicted_rmse :
    L=L+1

    if L==1:

        mul=np.dot(Train_features,new_a)
        sum=mul+new_b
        activation=relu(sum)
        beta_overall=np.dot(np.linalg.pinv(activation),E)
        y=np.dot(activation,beta_overall)
        E=E-y
        predicted_rmse=RMSE(y,Train_Label)
        print("-------- TRAINING RMSE ----------")
        print("")
        print(predicted_rmse,"With", L," Neuron")
        
    else:

        
        optimizer = ps.single.GlobalBestPSO(n_particles=20,dimensions=9, options=options, bounds=bounds)
        cost, pos = optimizer.optimize(swarm_initializer, iters=50,E=E)
        print(pos)
        a=pos[:8].reshape(len(Train_features[0]),1)

#         a=np.random.uniform(low=-1,high=1,size=[len(Train_features[0]),1]) #creating a new random weights for the second neuron and so on
#         b=np.random.uniform(low=-1,high=1,size=[1]) #creating a new random bias for the second neuron and so on
        b=pos[-1].reshape(1,)
        new_a=np.append(new_a,a,axis=1)
        mul=np.dot(Train_features,a)
        new_b=np.append(new_b,b,axis=0)
        sum=mul+b
        activation=relu(sum)
        
        beta=np.dot(pinv2(activation), E)
        y=np.dot(activation,beta)
        E=E-y
        beta_overall=np.append(beta_overall,beta,axis=0)
        predicted_rmse=RMSE(y,Train_Label)
        print(predicted_rmse,"with", L," Neuron ")
end_time=round(time.time()-start_time,3)
print("")
print("TRAINING TIME",end_time)


2020-10-30 22:02:40,104 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:  32%|████████████████████▏                                          |16/50, best_cost=0.18

[[-0.66107378]
 [-0.32275013]
 [-0.01174512]
 [-0.64901455]
 [-0.68621937]
 [ 0.01196192]
 [-0.0977581 ]
 [ 0.56662726]]
-------- TRAINING RMSE ----------

0.45298840411497093 With 1  Neuron


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=0.166
2020-10-30 22:02:40,426 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.16648653045108894, best pos: [ 0.32740918  0.35734395 -0.67814508 -0.4454094   0.53803603  0.13732783
 -0.26347853  0.57193597 -0.86541839]
2020-10-30 22:02:40,439 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:  34%|█████████████████████                                         |17/50, best_cost=0.138

[ 0.32740918  0.35734395 -0.67814508 -0.4454094   0.53803603  0.13732783
 -0.26347853  0.57193597 -0.86541839]
0.43067502740785935 with 2  Neuron 


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=0.138
2020-10-30 22:02:40,765 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.1375174565345297, best pos: [-0.30644588  0.93744598 -0.51980922 -0.99189073 -0.27648582 -0.53916074
  0.23366358  0.51934679 -0.61458546]
2020-10-30 22:02:40,778 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:  34%|█████████████████████                                         |17/50, best_cost=0.137

...

[-0.15492303  0.86423663 -0.60801103 -0.5050141   0.24459375  0.7586288
 -0.80179956 -0.76874332  0.18296874]
0.4590724232623783 with 48  Neuron 


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=0.434
2020-10-30 22:02:57,484 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.4340327587207026, best pos: [ 0.65666555  0.7333736   0.44361431  0.90246534 -0.829208    0.57174935
  0.44453707  0.76880554  0.31695109]
2020-10-30 22:02:57,501 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:  22%|█████████████▋                                                |11/50, best_cost=0.444

[ 0.65666555  0.7333736   0.44361431  0.90246534 -0.829208    0.57174935
  0.44453707  0.76880554  0.31695109]
0.44840509554793545 with 49  Neuron 


pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████████|50/50, best_cost=0.444
2020-10-30 22:02:57,906 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.44359635366457806, best pos: [ 0.39452492 -0.09040342  0.79437819  0.41082025 -0.8434031   0.25276226
 -0.4728907   0.38615299  0.34895006]


[ 0.39452492 -0.09040342  0.79437819  0.41082025 -0.8434031   0.25276226
 -0.4728907   0.38615299  0.34895006]
0.455772501230721 with 50  Neuron 

TRAINING TIME 17.816


In [8]:
#TESTING

print("-------- TESTING RMSE --------")
print("")
start_time=time.time()
h_test=np.dot(Test_features,new_a)
sum_test=h_test+new_b
activation_test=relu(sum_test)
y_test=np.dot(activation_test,beta_overall)
testing_rmse=RMSE(y_test,Test_Label)
print(testing_rmse)
end_time=round(time.time()-start_time,3)
print("")
print("TESTING TIME",end_time)



-------- TESTING RMSE --------

0.19595105287909553

TESTING TIME 0.002
